# Train a model and use it for prediction

Before running this notebook, run the auto-ml-sql-setup.ipynb notebook.

In [2]:
USE [automl]
GO

In [2]:
-- This shows using the AutoMLTrain stored procedure to create a forecasting model for the nyc_energy dataset.

INSERT INTO dbo.aml_model(RunId, ExperimentName, Model, LogFileText, WorkspaceName)
EXEC dbo.AutoMLTrain @input_query='
SELECT CAST(timeStamp as NVARCHAR(30)) as timeStamp,
       demand,
	   precip,
	   temp,
	   CASE WHEN timeStamp < ''2017-01-01'' THEN 0 ELSE 1 END AS is_validate_column
FROM nyc_energy
WHERE demand IS NOT NULL AND precip IS NOT NULL AND temp IS NOT NULL
and timeStamp < ''2017-02-01''',
@label_column='demand',
@task='forecasting',
@iterations=10,
@iteration_timeout_minutes=5,
@time_column_name='timeStamp',
@is_validate_column='is_validate_column',
@experiment_name='automl-sql-forecast',
@primary_metric='normalized_root_mean_squared_error'

In [3]:
-- This shows using the AutoMLPredict stored procedure to predict using a forecasting model for the nyc_energy dataset.

DECLARE @Model NVARCHAR(MAX) = (SELECT TOP 1 Model FROM dbo.aml_model
                                WHERE ExperimentName = 'automl-sql-forecast'
								ORDER BY CreatedDate DESC)

EXEC dbo.AutoMLPredict @input_query='
SELECT CAST(timeStamp AS NVARCHAR(30)) AS timeStamp,
       demand,
	   precip,
	   temp
FROM nyc_energy
WHERE demand IS NOT NULL AND precip IS NOT NULL AND temp IS NOT NULL
AND timeStamp >= ''2017-02-01''',
@label_column='demand',
@model=@model
WITH RESULT SETS ((timeStamp NVARCHAR(30), actual_demand FLOAT, precip FLOAT, temp FLOAT, predicted_demand FLOAT))

In [4]:
-- This lists all the metrics for all iterations for the most recent training run.

DECLARE @RunId NVARCHAR(43)
DECLARE @ExperimentName NVARCHAR(255)

SELECT TOP 1 @ExperimentName=ExperimentName, @RunId=SUBSTRING(RunId, 1, 43)
FROM aml_model
ORDER BY CreatedDate DESC

EXEC dbo.AutoMLGetMetrics @RunId, @ExperimentName